In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_percentage_error
from deap import base, creator, tools, algorithms

In [2]:
SEED = 42
np.random.seed(SEED)

data = pd.read_csv('DATA_DESA.csv')

df = pd.DataFrame(data)

# Normalisasi MinMax
scaler = MinMaxScaler()
df['jumlah_kecelakaan'] = scaler.fit_transform(df[['jumlah_kecelakaan']])

# Menambahkan kolom lag (pergeseran)
df['jumlah_kecelakaan_lag'] = df['jumlah_kecelakaan'].shift(1)
df = df.dropna() # Menghapus baris dengan nilai NaN

In [3]:
# Memisahkan data latih dan data uji sesuai rasio (0.8 untuk 80:20)
train_size = int(0.7 * len(df))
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

# Memisahkan fitur dan target untuk data latih
X_train = train_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_train = train_data['jumlah_kecelakaan'].values

# Memisahkan fitur dan target untuk data uji
X_test = test_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_test = test_data['jumlah_kecelakaan'].values

In [4]:
# Define evaluation function
def evaluate(individual):
    C, epsilon, gamma = individual
    if C <= 0 or epsilon <= 0 or gamma <= 0:
        return float('inf'),
    svr = SVR(C=C, epsilon=epsilon, gamma=gamma)
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    return mape,
    
# Setup DEAP framework
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Parameter ranges
parameter_ranges = [(10, 50), (0.001, 1), (0.001, 5)]

# Register genetic operators
toolbox.register("attr_float", random.uniform)
toolbox.register("individual", tools.initCycle, creator.Individual, 
                 (lambda: random.uniform(*parameter_ranges[0]),
                  lambda: random.uniform(*parameter_ranges[1]),
                  lambda: random.uniform(*parameter_ranges[2])), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxUniform, indpb=0.1)  # Menggunakan crossover uniform
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)  # Menggunakan mutasi flipbit
toolbox.register("select", tools.selTournament, tournsize=3) # Menggunakan seleksi turnamen
toolbox.register("evaluate", evaluate)

In [5]:
# Initialize population
population_size = 40
population = toolbox.population(n=population_size)

# Genetic algorithm parameters
crossover_rate = 1
mutation_rate = 0
num_generations = 250

# Run genetic algorithm
for generation in range(num_generations):
    print(f"Generasi {generation + 1}:")
    offspring = algorithms.varAnd(population, toolbox, cxpb=crossover_rate, mutpb=mutation_rate)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
        print(f"Individu: {ind}, MAPE: {fit[0]}")
    
    population = toolbox.select(offspring, k=len(population))

# Get the best individual
best_individual = tools.selBest(population, k=1)[0]
best_mape = best_individual.fitness.values[0]
C, epsilon, gamma = best_individual

# Test the best SVR model
best_svr = SVR(kernel='rbf', C=C, epsilon=epsilon, gamma=gamma)
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100

print("\nParameter terbaik:")
print(f"C: {C}")
print(f"Epsilon: {epsilon}")
print(f"Gamma: {gamma}")
print(f"MAPE terbaik: {best_mape}")

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)

Generasi 1:
Individu: [12.234529222018363, 0.6197512454446219, 0.1677273394929135], MAPE: 49.03560405882388
Individu: [25.946988536040735, 0.2595583021101699, 3.2428045168810353], MAPE: 45.83626278744008
Individu: [47.351761107414056, 0.44984119406890477, 2.6757403051768693], MAPE: 49.03560405882388
Individu: [22.116231878263232, 0.3716327304272957, 4.981397346998758], MAPE: 45.24668336222979
Individu: [14.122555673389892, 0.3868073840223577, 3.30525761917115], MAPE: 46.35962039519028
Individu: [27.46144078074595, 0.0744925634182496, 3.5239280665603094], MAPE: 46.877762896460645
Individu: [39.16053444126269, 0.9657885907421108, 3.015986450015052], MAPE: 49.03560405882388
Individu: [48.80756549664599, 0.5633742908816343, 4.925372705770434], MAPE: 49.03560405882388
Individu: [35.00062704455508, 0.09563531908767268, 3.9260954186333032], MAPE: 46.632801947915375
Individu: [22.648471773149232, 0.20709500795413147, 4.462733245737237], MAPE: 45.96286217057386
Individu: [46.160143783476755, 0.

In [6]:
# Nilai minimum dan maksimum asli dari data yang digunakan untuk normalisasi
data_min = 0  # Sesuaikan dengan nilai minimum data asli Anda
data_max = 20  # Sesuaikan dengan nilai maksimum data asli Anda

# Denormalisasi secara manual
y_pred_denormalized = y_pred * (data_max - data_min) + data_min

#pred_denormalized =  MinMaxScaler.inverse_transform(y_pred.reshape(1, -1))
print(y_pred_denormalized)

[12.67656778 11.15519816 11.15519816 11.15519816 11.65999623 10.75708616
 11.84697196 10.91658077 10.50032593  7.31638274  8.45732052 11.84697196
 10.18094894 12.67656778 11.15519816  4.7922895  11.84697196 11.65999623]
